In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.options.display.float_format = "{:.3f}".format

In [12]:
def seed_read_results(env_name, files, keys, seeds, update=True):
    parent = "results_chessworld/" + env_name + "/"
    final_dfs = {}

    for seed in seeds:
        all_dfs = []
        for file in files:

            # For GCN run on the correct curriculum on seed 5, replacing old incorrect curriculum
            if seed == 5 and file == 'ChessWorld-v1_gcn_formula_update2' and update:
                cur_name = parent + str(seed) + '/ChessWorld-v1_gcn_formula_update_quick.csv'

            else:
                cur_name = parent + str(seed) + "/" + file + ".csv"

            

            cur_df = pd.read_csv(cur_name)
            cur_df.set_index(["Task Set", "Task ID"], inplace=True)

            if "Successes x/29" in cur_df.columns:
                cur_df["Successes x/29"] *= (float(100)/29)
    
            if "Avg Discounted Return" in cur_df.columns:
                # cur_df.drop(columns=["Avg Discounted Return"], inplace=True)
                cur_df.drop(columns=["Avg Steps"], inplace=True)
    
            all_dfs.append(cur_df)
    
        df_concat = pd.concat(all_dfs, axis=1, keys=keys)
        final_dfs[seed] = df_concat

    df_mean = sum(final_dfs.values()) / len(final_dfs)
    df_std = sum([(df_mean - df_cur)**2 / len(final_dfs) for df_cur in final_dfs.values()]).apply(np.sqrt)
    return final_dfs, df_mean, df_std

In [13]:
def actual_stds(final_dfs, df_mean):
    df_mean_grouped = df_mean.groupby("Task Set").mean()
    final_dfs_grouped = [(cur_df.groupby("Task Set").mean() - df_mean_grouped)**2/len(final_dfs) for cur_df in final_dfs]

    df_std = sum(final_dfs_grouped).apply(np.sqrt)

    return df_std

In [14]:
files_finite_seeds = [
                'ChessWorld-v1_deepsets_formula_update',
                'ChessWorld-v1_gcn_formula_update2',
                'ChessWorld-v1_transformer_formula_update'
               ]

keys_seeds = ["DeepLTL", "LTL-GNN", "LTL-ENC"]

dfs_ra, df_ra_mean, df_ra_std = seed_read_results("ChessWorld-v1", files_finite_seeds, keys_seeds, list(range(1, 6)))

# df_ra_mean

In [15]:
df_ra_mean.groupby("Task Set").mean()

DeepLTL                              LTL-GNN  \
                        Successes x/29 Avg Discounted Return Successes x/29   
Task Set                                                                      
Avoid 2                         92.299                 0.886         95.172   
Avoid 3                         68.793                 0.658         82.586   
Avoid 4                         91.172                 0.886         92.690   
Avoid 5                         67.126                 0.642         74.253   
Reach                           99.080                 0.906         99.310   
Reach Avoid General             91.527                 0.873         93.596   
Reach Avoid X and not Y         91.862                 0.897         91.034   

                                                     LTL-ENC  \
                        Avg Discounted Return Successes x/29   
Task Set                                                       
Avoid 2                                 0.913         57.586   
Avoid 3                                 0.785         51.724   
Avoid 4                                 0.902         76.966   
Avoid 5                                 0.709         41.724   
Reach                                   0.915         70.575   
Reach Avoid General                     0.892         76.552   
Reach Avoid X and not Y                 0.888         80.828   

                                               
                        Avg Discounted Return  
Task Set                                       
Avoid 2                                 0.554  
Avoid 3                                 0.494  
Avoid 4                                 0.746  
Avoid 5                                 0.398  
Reach                                   0.642  
Reach Avoid General                     0.724  
Reach Avoid X and not Y                 0.785

In [16]:
actual_stds(dfs_ra.values(), df_ra_mean)

DeepLTL                              LTL-GNN  \
                        Successes x/29 Avg Discounted Return Successes x/29   
Task Set                                                                      
Avoid 2                          3.770                 0.037          3.243   
Avoid 3                          3.055                 0.030          6.597   
Avoid 4                          2.738                 0.027          3.867   
Avoid 5                          5.310                 0.050          8.520   
Reach                            1.839                 0.017          0.920   
Reach Avoid General              2.319                 0.023          1.946   
Reach Avoid X and not Y          1.337                 0.014          3.235   

                                                     LTL-ENC  \
                        Avg Discounted Return Successes x/29   
Task Set                                                       
Avoid 2                                 0.029         16.431   
Avoid 3                                 0.061         20.982   
Avoid 4                                 0.036         15.425   
Avoid 5                                 0.078         18.052   
Reach                                   0.009         20.913   
Reach Avoid General                     0.016         17.134   
Reach Avoid X and not Y                 0.031         11.132   

                                               
                        Avg Discounted Return  
Task Set                                       
Avoid 2                                 0.159  
Avoid 3                                 0.200  
Avoid 4                                 0.153  
Avoid 5                                 0.171  
Reach                                   0.187  
Reach Avoid General                     0.162  
Reach Avoid X and not Y                 0.110

In [17]:
files_infinite_seeds = [
                'ChessWorld-v1_deepsets_formula_update_inf',
                'ChessWorld-v1_gcn_formula_update_inf',
                'ChessWorld-v1_transformer_formula_update_inf'
               ]

keys_seeds = ["DeepLTL", "LTL-GNN", "LTL-ENC"]

dfs_rs, df_rs_mean, df_rs_std = seed_read_results("ChessWorld-v1", files_infinite_seeds, keys_seeds, list(range(1, 6)))

# df_ra_mean

In [18]:
# Last task mistakenly repeated
df_rs_m1 = df_rs_mean.iloc[:-1]

# df_rs_m1
df_rs_m1.groupby("Task Set").mean()

,DeepLTL,LTL-GNN,LTL-ENC
,Success rate,Success rate,Success rate
Task Set,,,
Reach avoid always eventually,0.957,0.928,0.676
Reach infinite training,0.400,0.860,0.431
Reach stay infinite,0.357,0.767,0.336


In [19]:
actual_stds([cdf.iloc[:-1] for cdf in dfs_rs.values()], df_rs_m1)

,DeepLTL,LTL-GNN,LTL-ENC
,Success rate,Success rate,Success rate
Task Set,,,
Reach avoid always eventually,0.017,0.093,0.285
Reach infinite training,0.490,0.280,0.422
Reach stay infinite,0.440,0.367,0.339
